# IBM Coursera Week 3 Applied Data Science Capstone

## Assignment Instructions

### In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project. For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset. Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.Your submission will be a link to your Jupyter Notebook on your Github repository.

#### link on how to create a web scraping HTML Tables with Python: https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [3]:
import requests
import lxml.html as lh
import pandas as pd

In [4]:
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
        
#Create a handle, page, to handle the contents of the website 

data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

doc = lh.fromstring(data.content)

tr_elements = doc.xpath('//tr')

In [5]:
#sanity check to ensure df is same width and column
[len(T) for T in tr_elements [:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [6]:
#Parse Table Header

tr_elements = doc.xpath('//tr')

list = []
i = 0 

for t in tr_elements[0]:
    i+=1
    name = t.text_content()
    print ('%d:"%s"'%(i,name))
    list.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


In [7]:
for y in range(1,len(tr_elements)):
    T=tr_elements[y]
    
    if len(T)!=3:
        break
        
    i=0
    
    for t in T.iterchildren():
        data=t.text_content()
        if i>0:
            try: 
                data = int(data)
            except:
                pass
        list[i][1].append(data)
        i+=1
        

In [8]:
#check to see if length and column are the same again
[len (C) for (title,C) in list]

[181, 181, 181]

#### Creating the Pandas data frame

In [9]:
Dict = {title:column for (title,column) in list}
df = pd.DataFrame(Dict)

In [10]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [11]:
df.columns = ['Postal Code', 'Borough', 'Neighbourhood']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

df.head()

,Neighbourhood,Postal Code,Borough
0,Not assigned\n,M1A\n,Not assigned\n
1,Not assigned\n,M2A\n,Not assigned\n
2,Parkwoods\n,M3A\n,North York\n
3,Victoria Village\n,M4A\n,North York\n
4,"Regent Park, Harbourfront\n",M5A\n,Downtown Toronto\n


#### Cleaning data frame

In [12]:
df = df.replace('\n', ' ', regex=True)
df.head()

,Neighbourhood,Postal Code,Borough
0,Not assigned,M1A,Not assigned
1,Not assigned,M2A,Not assigned
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,"Regent Park, Harbourfront",M5A,Downtown Toronto


#### Removing not assigned from Borough

In [13]:
NA = df[df['Borough'] == 'Not assigned '].index
df.drop(NA, inplace = True)
df.head()

,Neighbourhood,Postal Code,Borough
2,Parkwoods,M3A,North York
3,Victoria Village,M4A,North York
4,"Regent Park, Harbourfront",M5A,Downtown Toronto
5,"Lawrence Manor, Lawrence Heights",M6A,North York
6,"Queen's Park, Ontario Provincial Government",M7A,Downtown Toronto


#### Grouping Neigbourhoods based on similar Postal Code and Borough

In [14]:
df = df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postal Code', 'Borough', 'Neighbourhood']
df.head()

,Postal Code,Borough,Neighbourhood
0,,Canadian postal codes,
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


#### Cleaning up the data frame to be neat and tidy

In [15]:
df = df.drop(df.index[0])

In [16]:
df.shape

(103, 3)

In [17]:
df.head(20)

,Postal Code,Borough,Neighbourhood
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
9,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
10,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [18]:
df.to_csv(r'df_final.csv')